In [5]:
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import time
import itertools
import re
import numpy as np
import os
import fnmatch
import matplotlib.pyplot as plt
import distance
from fuzzysearch import find_near_matches

In [132]:
Lib_Info = pd.read_csv('~/Dropbox/xenoMAGE/Libraries/Lib_Info.csv',index_col=2)
barcode_to_protein = dict(zip(Lib_Info.Barcode,Lib_Info.Protein))
protein_to_family = dict(zip(Lib_Info.Protein,Lib_Info.Family))

Read in some initial data:

# 8/20/18 - TW14 - Initial read of SSAP and SSAP/SSB libraries in pET-DEST vectors along with early rounds of selection in EC and LL. 
  
Ran v3 150 with only F read (150bp). Got ~9.3 M reads (underclustered). The library was amplified with primers that bind before the barcode, and standard reverse primers for the plasmid. Amplicons were short for pJP005 and pARC8, but significantly longer for p444 as I only had long-distance primers.

Samples 42 - 69 = SSAP lib in pARC8 selections 1 to 6  
Samples 86,87,88 = initial SSAP libraries in p444-DEST, pJP005-DEST, and pARC8-DEST in host organisms  


# 9/10/18 - TW15 - Finished EC SSAP/Dual selections, LL SSAP selection, more rounds on LL Dual, a few initial rounds on myco.
  
Ran v3 150 with only F read (150bp). Got ~15.4 M reads (underclustered). The library was amplified with primers that bind before the barcode, and standard reverse primers for the plasmid. Amplicons were short for pJP005 and pARC8, but significantly longer for p444 as I only had long-distance primers.

Samples 33 -  48 = SSAP lib in pARC8 selections 7 to 10  

In [10]:
steplist = ['0','1','2','3','4','5','5p','6','7','8','9','10']

tw14Initial = [88]
tw14 = [42,69]
tw15 = [33,48]

replicates = ['1','2','3','-']
preSequence = 'CTTCCGATCTC'
editing = {'positive':'CCCG','negative':'GCCG'}

# Find files in TW14 folder which have been merged from multiple sequencing runs and create list
merge_list = []
files = os.listdir('./TW14')  
pattern = "*merge.fastq"  
for entry in files:  
    if fnmatch.fnmatch(entry, pattern):
        merge_list.append(int(re.findall('\d+',entry)[1]))

barcodes = {}
barcodes2 = {}
barcode_pairs = {}

a = 0
b = 0

samples = [(m,14) for m in tw14Initial]
samples += [(m,14) for m in range(tw14[0],tw14[1]+1)]
samples += [(m,15) for m in range(tw15[0],tw15[1]+1)]


for i in samples:
    if a < 1:
        sample_name = steplist[a]
    else:
        sample_name = steplist[a] + '_' + replicates[b]

    # clock the process time for each sample
    print('processing sample',sample_name,'..')
    t0 = time.clock()

    # initialize tracking variables
    count = 0
    junk = 0
    no_match = 0

    # read-in fastQ files using BioPython
    if i[1] == 14:
        if i[0] in merge_list:
            file = 'TW14/trimmed_TW14-%s_R1_merge.fastq' % str(i[0]).zfill(2)
        else:
            file = 'TW14/trimmed_TW14-%s_R1.fastq' % str(i[0]).zfill(2)
    elif i[1] == 15:
        file = 'TW15/trimmed_TW15-%s_R1.fastq' % str(i[0]).zfill(2)

    trimmed = SeqIO.parse(file,'fastq')

    # initialize tracking variable
    barcodes[sample_name] = []

    # iterate through reads
    for j in trimmed:
        count += 1
        match = ''
        match2 = ''

        # quality filter any reads shorter than 50-nt
        seq = str(j.seq)
        seqlen = len(seq)
        if seqlen > 60 and not re.search('[^ACTG]',seq):

            # match to an 11-nt sequence 1 bp before the beginning of the varied region.
            match = seq.find(preSequence)

            # if the priming sequence(s) are found, record barcode(s), otherwise count as junk.
            if match > 0:
                barcode = seq[match+12 : match+24]
                barcodes[sample_name].append(barcode)
            else:
                no_match += 1
        else:
            junk += 1

    # report out time and statistics
    t1 = time.clock()
    print('took ',t1-t0,'seconds to process',sample_name,'which contains: \n',count,'forward reads',
          '\n',junk,'of which were binned as junk, and \n',no_match,'of which had no match.')

    # iterate counter
    b += 1         
    if a == 0:
        a = 1
        b = 0
    if a == -1:
        a = 0
        b = 0
    if b == len(replicates):
        b = 0
        a += 1

processing sample 0 ..
took  0.5532839999999997 seconds to process 0 which contains: 
 21752 forward reads 
 109 of which were binned as junk, and 
 2472 of which had no match.
processing sample 1_1 ..
took  1.099501 seconds to process 1_1 which contains: 
 41870 forward reads 
 228 of which were binned as junk, and 
 971 of which had no match.
processing sample 1_2 ..
took  0.1686230000000002 seconds to process 1_2 which contains: 
 6441 forward reads 
 420 of which were binned as junk, and 
 268 of which had no match.
processing sample 1_3 ..
took  0.8291700000000004 seconds to process 1_3 which contains: 
 32296 forward reads 
 213 of which were binned as junk, and 
 762 of which had no match.
processing sample 1_- ..
took  1.2615570000000007 seconds to process 1_- which contains: 
 49491 forward reads 
 336 of which were binned as junk, and 
 6155 of which had no match.
processing sample 2_1 ..
took  0.8513390000000003 seconds to process 2_1 which contains: 
 33147 forward reads 
 

In [166]:
# combine barcodes into dataframe, count their occurrences, separate data by experiment, sort and filter data,
# and export data as an excel file.

n = 35 # filter out any single barcodes counted fewer than n times in aggregate

# First count the number of times each barcode is seen in every experiment

barcodes = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in barcodes.items() ]))
counts = pd.DataFrame()
for c in barcodes.columns:
    count = barcodes[c].value_counts()
    count = pd.DataFrame(count,columns=[c])
    counts = pd.concat([counts,count], axis=1)

sums = counts.sum(axis=1)
sums = sums[sums > n]
counts = counts[counts.index.isin(sums.index)]

# Then assign a library number to each barcode and sort dataframes by library number and then the initial count
counts.insert(loc=0, column='SSAP', value=[None]*counts.shape[0])
for i in counts.index.tolist():
    try:
        counts.loc[i,'SSAP'] = barcode_to_protein[i]
    except:
        continue
counts = counts.sort_values(by=['SSAP','0'],ascending=[True,False])

# Search unmatched barcodes to see if they are one mutation away from only one known barcode
# Add unmatched count to known barcode's count in that case. Otherwise drop unmatched barcode.

for mutant in counts[counts.SSAP.isnull()].index:
    neighbors = []
    for barcode in barcode_to_protein.keys():
        if distance.levenshtein(mutant,barcode) == 1:
            neighbors.append(barcode)
    if len(neighbors) == 1:
        new_count = counts.loc[neighbors[0]].drop('SSAP').fillna(0) + counts.loc[mutant].drop('SSAP').fillna(0)
        new_count['SSAP'] = counts.loc[neighbors[0]].SSAP
        counts.loc[neighbors[0]] = new_count
    counts = counts.drop(mutant)

# Remove non-SEER members

counts = counts.set_index('SSAP').loc[[x for x in counts.SSAP if x[0:2] == 'SR']]

# Then assign family to each SSAP
counts.insert(loc=0, column='Family', value=[None]*counts.shape[0])
for i in counts.index.tolist():
    try:
        counts.loc[i,'Family'] = protein_to_family[i]
    except:
        continue

# Report out missing members

print('missing SEER members:\n',
      [y for y in [x for x in barcode_to_protein.values() if x[:2] == 'SR'] if y not in counts.index])

missing SEER members:
 ['SR073', 'SR125']


In [168]:
selections = 10
replicates = 3
# Minimum frequency in negative control for downstream analylsis
minimum = 0.0005

for sel in range(1,selections+1):
    df = pd.DataFrame()
    dg = pd.DataFrame()
    
    negative = str(sel) + '_-'
    dg['neg_frequency'] = counts[negative] / counts[negative].sum()
    dg['threshold'] = dg['neg_frequency'].apply(lambda x: 1 if x >= minimum else 0)
    
    # Calculate enrichment relative to non-selective control
    for rep in range(1,replicates+1):
        handle = str(sel) + '_' + str(rep)
        df[handle] = counts[handle] / counts[handle].sum()
    
    # Record and remove enrichment values from analysis for those with low denominator in negative run.
    counts['enrichment_'+str(sel)] = df.sum(axis=1) / dg['neg_frequency'] * dg.threshold
    
    # Record average frequency values for selective replicates
    counts['frequency_'+str(sel)] = df.sum(axis=1) / 3

# Re-order columns
counts = counts.sort_index(axis=1)

#Write out to Excel
writer = pd.ExcelWriter('SR016.xlsx', engine='xlsxwriter')
counts.sort_values('enrichment_10',ascending=False).to_excel(writer, sheet_name = 'SEER')
counts.groupby(['Family']).sum().to_excel(writer, sheet_name = 'Family Stats')
writer.save()

Efficiencies of best members...TW20

In [8]:
tally = pd.DataFrame(columns=['sample','replicate','codon_counts'])
file = 'TW20/trimmed_TW20-%s_R1.fastq'
SSAPs = ['PF020','SR011','SR016','SR055','SR085','SR095','SR120']
pre_seq = 'TAGCTAAAAC'

s = 0
rep = 1
for i in range(17,38):
    t0 = time.clock()

    f = file % (str(i).zfill(2))
    trimmed = SeqIO.parse(f,'fastq')

    log = ['INIT']
    count = 0
    junk = 0
    for r in trimmed:
        # match to a 10-nt sequence before the edited region, allowing one mismatch or indel.
        # quality filter any reads shorter than 50-nt.
        seq = str(r.seq)
        if len(seq) > 50:
            match = find_near_matches(pre_seq,seq,max_l_dist=1)
        if match:
            log.append(seq[match[0][1]:match[0][1]+4])
        else:
            junk += 1

        count += 1
    df = pd.DataFrame(log)
    df.columns = ['sequence']

    tally.loc[len(tally)] = [SSAPs[s],rep,df.sequence.value_counts()]

    print(round(time.clock() - t0,1),'seconds process time for loop',SSAPs[s],'-',rep,'with',count,'reads',
          'of which',junk,'are junk.')

    s += 1
    if s == 7:
        s = 0
        rep += 1

5.3 seconds process time for loop PF020 - 1 with 48126 reads of which 2891 are junk.
12.6 seconds process time for loop SR011 - 1 with 111170 reads of which 3624 are junk.
14.4 seconds process time for loop SR016 - 1 with 128740 reads of which 12357 are junk.
7.4 seconds process time for loop SR055 - 1 with 64494 reads of which 6204 are junk.
2.8 seconds process time for loop SR085 - 1 with 25237 reads of which 2071 are junk.
5.1 seconds process time for loop SR095 - 1 with 44676 reads of which 2568 are junk.
2.8 seconds process time for loop SR120 - 1 with 23966 reads of which 336 are junk.
4.8 seconds process time for loop PF020 - 2 with 42087 reads of which 3293 are junk.
15.2 seconds process time for loop SR011 - 2 with 143240 reads of which 19829 are junk.
24.9 seconds process time for loop SR016 - 2 with 219928 reads of which 10955 are junk.
13.9 seconds process time for loop SR055 - 2 with 123317 reads of which 7715 are junk.
15.8 seconds process time for loop SR085 - 2 with 136

In [9]:
analysis = pd.DataFrame(columns=['sample','replicate','positive','negative','junk','efficiency','junk_rate'])
editing_ref = {'positive':'CCCG','negative':'GCCG'}

for bug in tally.keys():
    for sample in tally['sample'].unique():
        data = tally[tally['sample'] == sample]
        for replicate in data['replicate']:
            for item in data[data['replicate'] == replicate].codon_counts:
                positive = negative = junk = efficiency = junk_rate = 0
                sample_name = sample + '_' + str(replicate)
                for i in item.index:
                    if i == editing_ref['negative']:
                        negative = item[i]
                    elif i == editing_ref['positive']:
                        positive = item[i]
                    else:
                        junk += item[i]
        
                if negative > 0 :
                    efficiency = positive/(positive+negative)
                junk_rate = junk/(positive+negative+junk)
                analysis.loc[sample_name] = [sample,replicate,positive,negative,junk,efficiency,junk_rate]
                print(bug,sample,'\npositive: ',positive,'\nnegative: ',negative,'\njunk: ',junk,'\nefficiency: ',efficiency,
                      '\njunk rate: ',junk_rate,'\n')

# write out to excel
writer = pd.ExcelWriter('SR016-efficiencies.xlsx', engine='xlsxwriter')
analysis.to_excel(writer, sheet_name = 'EC Winners')
writer.save()

sample PF020 
positive:  8311 
negative:  36687 
junk:  238 
efficiency:  0.184697097649 
junk rate:  0.00526129631267 

sample PF020 
positive:  5887 
negative:  32743 
junk:  165 
efficiency:  0.152394512037 
junk rate:  0.00425312540276 

sample PF020 
positive:  18406 
negative:  101724 
junk:  614 
efficiency:  0.153217347873 
junk rate:  0.00508513880607 

sample SR011 
positive:  24853 
negative:  82189 
junk:  505 
efficiency:  0.232179892005 
junk rate:  0.00469562144923 

sample SR011 
positive:  21989 
negative:  100662 
junk:  761 
efficiency:  0.179281049482 
junk rate:  0.00616633714712 

sample SR011 
positive:  13473 
negative:  49100 
junk:  308 
efficiency:  0.215316510316 
junk rate:  0.00489814093287 

sample SR016 
positive:  34601 
negative:  81005 
junk:  778 
efficiency:  0.299301074339 
junk rate:  0.00668476766566 

sample SR016 
positive:  62059 
negative:  145666 
junk:  1249 
efficiency:  0.298755566253 
junk rate:  0.00597682008288 

sample SR016 
positive

## Beta v. PapRecT v. PF020 efficiency tests with varying oligo and inducer conc.

### TW21: PapRecT v. Beta with varied arabinose
  
pARC8 RBS and original Beta codons (not SR085 codons).

In [7]:
tally = pd.DataFrame(columns=['ssap','inducer','replicate','codon_counts'])
file = 'TW21/trimmed_TW21-%s_R1.fastq'
SSAPs = ['SR016','Beta']
inducer = ['.01%','.05%','.1%','.2%','.4%']
pre_seq = 'TAGCTAAAAC'

s = 0
i = 0
rep = 1
for j in range(1,31):
    t0 = time.clock()

    f = file % (str(j).zfill(2))
    trimmed = SeqIO.parse(f,'fastq')

    log = ['INIT']
    count = 0
    junk = 0
    for r in trimmed:
        # match to a 10-nt sequence before the edited region, allowing one mismatch or indel.
        # quality filter any reads shorter than 50-nt.
        seq = str(r.seq)
        if len(seq) > 50:
            match = find_near_matches(pre_seq,seq,max_l_dist=1)
        if match:
            log.append(seq[match[0][1]:match[0][1]+4])
        else:
            junk += 1

        count += 1
    df = pd.DataFrame(log)
    df.columns = ['sequence']

    tally.loc[len(tally)] = [SSAPs[s],inducer[i],rep,df.sequence.value_counts()]

    print(round(time.clock() - t0,1),'seconds process time for loop',SSAPs[s],'-',inducer[i],'-',
          rep,'with',count,'reads','of which',junk,'are junk.')
    i += 1
    if i == 5:
        i = 0
        s += 1
        if s == 2:
            s = 0
            rep += 1
            
analysis = pd.DataFrame(columns=['ssap','inducer','replicate','positive','negative','junk','efficiency','junk_rate'])
editing_ref = {'positive':'CCCG','negative':'GCCG'}

for ssap in tally['ssap'].unique():
    for inducer in tally['inducer'].unique():
        data = tally[(tally['ssap'] == ssap) & (tally['inducer'] == inducer)]
        for replicate in data['replicate']:
            for item in data[data['replicate'] == replicate].codon_counts:
                positive = negative = junk = efficiency = junk_rate = 0
                sample_name = ssap + '_' + inducer + '_' + str(replicate)
                for i in item.index:
                    if i == editing_ref['negative']:
                        negative = item[i]
                    elif i == editing_ref['positive']:
                        positive = item[i]
                    else:
                        junk += item[i]

                if negative > 0 :
                    efficiency = positive/(positive+negative)
                junk_rate = junk/(positive+negative+junk)
                analysis.loc[sample_name] = [ssap,inducer,replicate,positive,negative,junk,efficiency,junk_rate]
                print(sample_name,'\npositive: ',positive,'\nnegative: ',negative,'\njunk: ',junk,
                      '\nefficiency: ',efficiency,'\njunk rate: ',junk_rate,'\n')

# write out to excel
writer = pd.ExcelWriter('SR016-efficienciesb.xlsx', engine='xlsxwriter')
analysis.to_excel(writer, sheet_name = 'SR016 v Beta')
writer.save()

10.0 seconds process time for loop SR016 - .01% - 1 with 82301 reads of which 14560 are junk.
6.6 seconds process time for loop SR016 - .05% - 1 with 39615 reads of which 509 are junk.
15.8 seconds process time for loop SR016 - .1% - 1 with 116199 reads of which 3745 are junk.
6.1 seconds process time for loop SR016 - .2% - 1 with 48226 reads of which 2364 are junk.
12.6 seconds process time for loop SR016 - .4% - 1 with 99954 reads of which 2848 are junk.
10.8 seconds process time for loop Beta - .01% - 1 with 89099 reads of which 6342 are junk.
6.9 seconds process time for loop Beta - .05% - 1 with 55148 reads of which 594 are junk.
8.2 seconds process time for loop Beta - .1% - 1 with 66643 reads of which 1873 are junk.
19.5 seconds process time for loop Beta - .2% - 1 with 158661 reads of which 4999 are junk.
9.5 seconds process time for loop Beta - .4% - 1 with 78055 reads of which 2844 are junk.
11.5 seconds process time for loop SR016 - .01% - 2 with 102558 reads of which 9704 a

### TW22 - Beta v. PapRecT varying oligo conc.

In [6]:
tally = pd.DataFrame(columns=['ssap','oligo','replicate','codon_counts'])
file = 'TW22/trimmed_TW22-%s_R1.fastq'
SSAPs = ['SR016','Beta']
oligo = ['0.1','0.5','1','5','10','20']
pre_seq = 'TAGCTAAAAC'

s = 0
o = 0
rep = 1
for j in range(1,37):
    t0 = time.clock()

    f = file % (str(j).zfill(2))
    trimmed = SeqIO.parse(f,'fastq')

    log = ['INIT']
    count = 0
    junk = 0
    for r in trimmed:
        # match to a 10-nt sequence before the edited region, allowing one mismatch or indel.
        # quality filter any reads shorter than 50-nt.
        seq = str(r.seq)
        if len(seq) > 50:
            match = find_near_matches(pre_seq,seq,max_l_dist=1)
        if match:
            log.append(seq[match[0][1]:match[0][1]+4])
        else:
            junk += 1

        count += 1
    df = pd.DataFrame(log)
    df.columns = ['sequence']

    tally.loc[len(tally)] = [SSAPs[s],oligo[o],rep,df.sequence.value_counts()]

    print(round(time.clock() - t0,1),'seconds process time for loop',SSAPs[s],'-',oligo[o],'-',
          rep,'with',count,'reads','of which',junk,'are junk.')
    o += 1
    if o == 6:
        o = 0
        s += 1
        if s == 2:
            s = 0
            rep += 1
            
analysis = pd.DataFrame(columns=['ssap','oligo','replicate','positive','negative','junk','efficiency','junk_rate'])
editing_ref = {'positive':'CCCG','negative':'GCCG'}


for ssap in tally['ssap'].unique():
    for oligo in tally['oligo'].unique():
        data = tally[(tally['ssap'] == ssap) & (tally['oligo'] == oligo)]
        for replicate in data['replicate']:
            for item in data[data['replicate'] == replicate].codon_counts:
                positive = negative = junk = efficiency = junk_rate = 0
                sample_name = ssap + '_' + oligo + '_' + str(replicate)
                for i in item.index:
                    if i == editing_ref['negative']:
                        negative = item[i]
                    elif i == editing_ref['positive']:
                        positive = item[i]
                    else:
                        junk += item[i]

                if negative > 0 :
                    efficiency = positive/(positive+negative)
                junk_rate = junk/(positive+negative+junk)
                analysis.loc[sample_name] = [ssap,inducer,replicate,positive,negative,junk,efficiency,junk_rate]
                print(sample_name,'\npositive: ',positive,'\nnegative: ',negative,'\njunk: ',junk,
                      '\nefficiency: ',efficiency,'\njunk rate: ',junk_rate,'\n')

# write out to excel
writer = pd.ExcelWriter('SR016-efficienciesc.xlsx', engine='xlsxwriter')
analysis.to_excel(writer, sheet_name = 'SR016 v Beta')
writer.save()

0.9 seconds process time for loop SR016 - 0.1 - 1 with 7507 reads of which 257 are junk.
1.5 seconds process time for loop SR016 - 0.5 - 1 with 9772 reads of which 403 are junk.
1.7 seconds process time for loop SR016 - 1 - 1 with 9470 reads of which 406 are junk.
1.9 seconds process time for loop SR016 - 5 - 1 with 12988 reads of which 735 are junk.
1.5 seconds process time for loop SR016 - 10 - 1 with 11224 reads of which 795 are junk.
1.5 seconds process time for loop SR016 - 20 - 1 with 9522 reads of which 592 are junk.
1.3 seconds process time for loop Beta - 0.1 - 1 with 10403 reads of which 457 are junk.
1.2 seconds process time for loop Beta - 0.5 - 1 with 10072 reads of which 526 are junk.
1.3 seconds process time for loop Beta - 1 - 1 with 10753 reads of which 394 are junk.
1.2 seconds process time for loop Beta - 5 - 1 with 10473 reads of which 562 are junk.
1.1 seconds process time for loop Beta - 10 - 1 with 9444 reads of which 462 are junk.
1.0 seconds process time for lo

### TW25 - PF020 varied oligo and arabinose

In [12]:
tally = pd.DataFrame(columns=['sample','codon_counts'])
file = 'TW25/trimmed_TW25-%s_R1.fastq'
pre_seq = 'TAGCTAAAAC'
samples = [i for j in [range(97,107),range(109,119),range(121,131)] for i in j]

for j in samples:
    t0 = time.clock()

    f = file % (str(j).zfill(2))
    trimmed = SeqIO.parse(f,'fastq')

    log = ['INIT']
    count = 0
    junk = 0
    for r in trimmed:
        # match to a 10-nt sequence before the edited region, allowing one mismatch or indel.
        # quality filter any reads shorter than 50-nt.
        seq = str(r.seq)
        if len(seq) > 50:
            match = find_near_matches(pre_seq,seq,max_l_dist=1)
        if match:
            log.append(seq[match[0][1]:match[0][1]+4])
        else:
            junk += 1

        count += 1
    df = pd.DataFrame(log)
    df.columns = ['sequence']

    tally.loc[len(tally)] = [j,df.sequence.value_counts()]

    print(round(time.clock() - t0,1),'seconds process time for loop',j,'with',count,'reads','of which',junk,'are junk.')

analysis = pd.DataFrame(columns=['sample','positive','negative','junk','efficiency','junk_rate'])
editing_ref = {'positive':'CCCG','negative':'GCCG'}

for sample in tally['sample'].unique():
    for item in tally[tally['sample'] == sample].codon_counts:
        positive = negative = junk = efficiency = junk_rate = 0
        
        for i in item.index:
            if i == editing_ref['negative']:
                negative = item[i]
            elif i == editing_ref['positive']:
                positive = item[i]
            else:
                junk += item[i]

        if negative > 0 :
            efficiency = positive/(positive+negative)
        junk_rate = junk/(positive+negative+junk)
        analysis.loc[sample] = [sample,positive,negative,junk,efficiency,junk_rate]
        print(sample,'\npositive: ',positive,'\nnegative: ',negative,'\njunk: ',junk,
              '\nefficiency: ',efficiency,'\njunk rate: ',junk_rate,'\n')
        
# write out to excel
writer = pd.ExcelWriter('SR016-efficienciesd.xlsx', engine='xlsxwriter')
analysis.to_excel(writer, sheet_name = 'PF020')
writer.save()


1.1 seconds process time for loop 97 with 8700 reads of which 701 are junk.
1.2 seconds process time for loop 98 with 9243 reads of which 551 are junk.
1.0 seconds process time for loop 99 with 8269 reads of which 572 are junk.
1.1 seconds process time for loop 100 with 8543 reads of which 576 are junk.
0.1 seconds process time for loop 101 with 973 reads of which 250 are junk.
1.3 seconds process time for loop 102 with 10523 reads of which 598 are junk.
1.3 seconds process time for loop 103 with 10827 reads of which 714 are junk.
1.0 seconds process time for loop 104 with 7958 reads of which 611 are junk.
1.1 seconds process time for loop 105 with 9033 reads of which 642 are junk.
0.9 seconds process time for loop 106 with 7554 reads of which 488 are junk.
1.0 seconds process time for loop 109 with 7751 reads of which 453 are junk.
1.1 seconds process time for loop 110 with 8547 reads of which 541 are junk.
1.3 seconds process time for loop 111 with 10649 reads of which 739 are junk.


### TW23 - PapRecT v. Beta (wiltype codons) in pARC8 with pJP RBS
to check if the pJP RBS caused PapRecT to work better than Beta, which was seen in the SEER library screen

In [13]:
tally = pd.DataFrame(columns=['ssap','replicate','codon_counts'])
file = 'TW23/trimmed_TW23-%s_R1.fastq'
SSAPs = ['PapRecT','Beta']
pre_seq = 'TAGCTAAAAC'

s = 0
rep = 1
for j in range(238,244):
    t0 = time.clock()

    f = file % (str(j).zfill(2))
    trimmed = SeqIO.parse(f,'fastq')

    log = ['INIT']
    count = 0
    junk = 0
    for r in trimmed:
        # match to a 10-nt sequence before the edited region, allowing one mismatch or indel.
        # quality filter any reads shorter than 50-nt.
        seq = str(r.seq)
        if len(seq) > 50:
            match = find_near_matches(pre_seq,seq,max_l_dist=1)
        if match:
            log.append(seq[match[0][1]:match[0][1]+4])
        else:
            junk += 1

        count += 1
    df = pd.DataFrame(log)
    df.columns = ['sequence']

    tally.loc[len(tally)] = [SSAPs[s],rep,df.sequence.value_counts()]

    print(round(time.clock() - t0,1),'seconds process time for loop',SSAPs[s],'-',
          rep,'with',count,'reads','of which',junk,'are junk.')

    rep += 1
    if rep == 4:
        rep = 1
        s += 1

FileNotFoundError: [Errno 2] No such file or directory: 'TW23/trimmed_TW23-238_R1.fastq'

In [5]:
analysis = pd.DataFrame(columns=['ssap','replicate','positive','negative','junk','efficiency','junk_rate'])
editing_ref = {'positive':'CCCG','negative':'GCCG'}


for ssap in tally['ssap'].unique():
    data = tally[(tally['ssap'] == ssap)]
    for replicate in data['replicate']:
        for item in data[data['replicate'] == replicate].codon_counts:
            positive = negative = junk = efficiency = junk_rate = 0
            sample_name = ssap + '_' + str(replicate)
            for i in item.index:
                if i == editing_ref['negative']:
                    negative = item[i]
                elif i == editing_ref['positive']:
                    positive = item[i]
                else:
                    junk += item[i]

            if negative > 0 :
                efficiency = positive/(positive+negative)
            junk_rate = junk/(positive+negative+junk)
            analysis.loc[sample_name] = [ssap,replicate,positive,negative,junk,efficiency,junk_rate]
            print(sample_name,'\npositive: ',positive,'\nnegative: ',negative,'\njunk: ',junk,
                  '\nefficiency: ',efficiency,'\njunk rate: ',junk_rate,'\n')


SR016_1 
positive:  170 
negative:  3989 
junk:  15 
efficiency:  0.0408752103871 
junk rate:  0.00359367513177 

SR016_2 
positive:  89 
negative:  3354 
junk:  12 
efficiency:  0.0258495498112 
junk rate:  0.00347322720695 

SR016_3 
positive:  758 
negative:  17398 
junk:  118 
efficiency:  0.0417492839833 
junk rate:  0.00645726168327 

Beta_1 
positive:  367 
negative:  12499 
junk:  79 
efficiency:  0.0285247940308 
junk rate:  0.00610274237157 

Beta_2 
positive:  376 
negative:  9832 
junk:  88 
efficiency:  0.0368338557994 
junk rate:  0.00854700854701 

Beta_3 
positive:  466 
negative:  15709 
junk:  94 
efficiency:  0.0288098918083 
junk rate:  0.00577785973323 



Next3 - SR016 v. Beta (wiltype codons) and both in pARC8 with pJP RBS

In [3]:
tally = pd.DataFrame(columns=['ssap','replicate','codon_counts'])
file = 'Next3/Data/trimmed/trimmed_S%i_merged_R1.fastq'
SSAPs = ['SR016','Beta']
pre_seq = 'TAGCTAAAAC'
samples = [111,116]

s = 0
rep = 1
for j in range(samples[0],samples[1]+1):
    t0 = time.clock()

    f = file % j
    trimmed = SeqIO.parse(f,'fastq')

    log = ['INIT']
    count = 0
    junk = 0
    for r in trimmed:
        # match to a 10-nt sequence before the edited region, allowing one mismatch or indel.
        # quality filter any reads shorter than 50-nt.
        seq = str(r.seq)
        if len(seq) > 50:
            match = find_near_matches(pre_seq,seq,max_l_dist=1)
        if match:
            log.append(seq[match[0][1]:match[0][1]+4])
        else:
            junk += 1

        count += 1
    df = pd.DataFrame(log)
    df.columns = ['sequence']

    tally.loc[len(tally)] = [SSAPs[s],rep,df.sequence.value_counts()]

    print(round(time.clock() - t0,1),'seconds process time for loop',SSAPs[s],'-',
          rep,'with',count,'reads','of which',junk,'are junk.')

    rep += 1
    if rep == 4:
        rep = 1
        s += 1

2.5 seconds process time for loop SR016 - 1 with 21449 reads of which 1889 are junk.
2.8 seconds process time for loop SR016 - 2 with 23730 reads of which 1937 are junk.
2.8 seconds process time for loop SR016 - 3 with 24081 reads of which 2026 are junk.
2.4 seconds process time for loop Beta - 1 with 21228 reads of which 2368 are junk.
2.5 seconds process time for loop Beta - 2 with 22117 reads of which 2062 are junk.
2.6 seconds process time for loop Beta - 3 with 22347 reads of which 1990 are junk.


In [4]:
analysis = pd.DataFrame(columns=['ssap','replicate','positive','negative','junk','efficiency','junk_rate'])
editing_ref = {'positive':'CCCG','negative':'GCCG'}


for ssap in tally['ssap'].unique():
    data = tally[(tally['ssap'] == ssap)]
    for replicate in data['replicate']:
        for item in data[data['replicate'] == replicate].codon_counts:
            positive = negative = junk = efficiency = junk_rate = 0
            sample_name = ssap + '_' + str(replicate)
            for i in item.index:
                if i == editing_ref['negative']:
                    negative = item[i]
                elif i == editing_ref['positive']:
                    positive = item[i]
                else:
                    junk += item[i]

            if negative > 0 :
                efficiency = positive/(positive+negative)
            junk_rate = junk/(positive+negative+junk)
            analysis.loc[sample_name] = [ssap,replicate,positive,negative,junk,efficiency,junk_rate]
            print(sample_name,'\npositive: ',positive,'\nnegative: ',negative,'\njunk: ',junk,
                  '\nefficiency: ',efficiency,'\njunk rate: ',junk_rate,'\n')



SR016_1 
positive:  1411 
negative:  18012 
junk:  138 
efficiency:  0.0726458322607 
junk rate:  0.00705485404632 

SR016_2 
positive:  1719 
negative:  19904 
junk:  171 
efficiency:  0.079498681959 
junk rate:  0.00784619620079 

SR016_3 
positive:  1367 
negative:  20531 
junk:  158 
efficiency:  0.0624257923098 
junk rate:  0.00716358360537 

Beta_1 
positive:  611 
negative:  18098 
junk:  152 
efficiency:  0.0326580789994 
junk rate:  0.00805895763745 

Beta_2 
positive:  1023 
negative:  18870 
junk:  163 
efficiency:  0.0514251244156 
junk rate:  0.00812724371759 

Beta_3 
positive:  853 
negative:  19339 
junk:  166 
efficiency:  0.0422444532488 
junk rate:  0.0081540426368 

